In [1]:
import torch #for creating tensor objects, backprop etc
import torch.nn.functional as F #for one_hot

In [2]:
word = [
    "aarav", "vivaan", "aditya", "vihaan", "ishaan", "arjun", "sai", "ayaan", "krishna", "rohan",
    "reyansh", "shiv", "mohit", "karthik", "lakshman", "siddharth", "manish", "vishal", "vivek", "rahul",
    "manav", "nikhil", "kunal", "anish", "yash", "abhinav", "pranav", "amit", "gaurav", "ravi",
    "raj", "surya", "harsh", "tushar", "akash", "parth", "raghav", "ramesh", "ankit", "suresh",
    "pritam", "prem", "himesh", "ashwin", "dhruv", "sandeep", "vikas", "ajay", "anil", "madhav",
    "deepak", "bhuvan", "ashok", "ram", "shivansh", "nitin", "saurabh", "udit", "shaurya", "manoj",
    "chirag", "kiran", "amitabh", "nashit", "siddhi", "ritika", "ananya", "priya", "aishwarya", "sneha",
    "radhika", "meera", "swati", "pooja", "shruti", "simran", "nisha", "sanya", "kavya", "madhuri",
    "neha", "jaya", "mitali", "sonali", "laxmi", "vidya", "komal", "shalini", "tanu", "shreya",
    "nupur", "isha", "rupa", "divya", "ritu", "vandana", "pragya", "suman", "deepika", "manju",
    "shweta", "vaishnavi", "parul", "gayatri", "aarti", "tanvi", "chhavi", "anju", "tanisha", "sakshi",
    "simran", "sonal", "ravina", "meenal", "aarti", "shivani", "ankita", "kiran", "yamini", "sonalika"
]

#A list of strings; chatgpt generated these kerala origin names to train the model

In [3]:
import string
chars=list(string.ascii_lowercase)
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i+1:s for i,s in enumerate(chars)}
itos[0]="."

#since indexing is based on integers from 0 to 1,2,3.... we map each alphabet to an integer in that order itself

In [4]:
x=[]
xi=[]
y=[]
yi=[]
for w in word:
  w='.'+w+'.'
  for i in zip(w,w[1:]):
    x.append(i[0])
    xi.append(stoi[i[0]])
    y.append(i[1])
    yi.append(stoi[i[1]])

#using zip we create bigraphs and thus generating i/p and o/p labels

In [5]:
xi=torch.tensor(xi)
yi=torch.tensor(yi)
#creating tensor objects

In [6]:
xenc = F.one_hot(xi, num_classes=27).float() #for more efficient working of model, we use one_hat for classes

In [7]:
W = torch.randn((27, 27),requires_grad=True) #since its a single layer nn with 27 neruons we need a 27x27 weight matrix

In [8]:
#forward pass matrix multiplication (we use @ for product)
logits=xenc@W
counts=logits.exp()
probs=counts/counts.sum(dim=1,keepdim=True)
probs.shape

#here we consider cross entropy loss, basically sum of negative logs of each predicted probability values corresponding to actual labels (we already have them for training set)
nlls=torch.zeros(len(yi))
j=0
for i in yi:
  loss=torch.log(probs[j,i])
  nlls[j]=-loss
  j+=1

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())
LOSS=nlls.mean()

average negative log likelihood, i.e. loss = 3.8396403789520264


In [9]:
W.grad=None
LOSS.backward()
#backprop to calculate gradients of each weights like how they affect the final loss

In [19]:
#conducting various epochs and using manual gradient descent to converge to min loss and get corresponding wieghts
alpha=1
for i in range(50):
  W.data+=-alpha*W.grad

  #forward pass
  logits=xenc@W
  counts=logits.exp()
  probs=counts/counts.sum(dim=1,keepdim=True)
  probs.shape

  nlls=torch.zeros(len(yi))
  j=0
  for i in yi:
    loss=torch.log(probs[j,i])
    nlls[j]=-loss
    j+=1

  print('=========')
  print('average negative log likelihood, i.e. loss =', nlls.mean().item()+0.01*(W**2).mean())
  LOSS=nlls.mean()+0.01*(W**2).mean()

average negative log likelihood, i.e. loss = tensor(2.7222, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7191, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7161, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7133, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7106, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7081, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7057, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7035, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.7014, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.6995, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.6977, grad_fn=<AddBackward0>)
average negative log likelihood, i.e. loss = tensor(2.6960, grad_fn=<AddBack

In [23]:
g = torch.Generator().manual_seed(2147483647)
for i in range(30):
  out =[]
  ix=0
  while True:
    inputencode=F.one_hot(torch.tensor([ix]),num_classes=27).float()
    logits=inputencode@W
    counts=logits.exp()
    p=counts/counts.sum(1,keepdims=True)

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(out)

  #testing yayyy !!


['a', '.']
['n', 'i', 'd', 'v', '.']
['s', 'h', 'i', '.']
['s', 'h', 'a', '.']
['p', '.']
['s', 'h', 'a', 'n', 'w', 'o', 'c', 'n', 'a', '.']
['s', 'h', 'i', '.']
['s', 'h', 't', 'o', 'h', 'i', 't', 'u', 's', 'h', 'i', '.']
['s', 'h', '.']
['s', 'h', 'a', 'n', 'a', 'n', 'u', 'z', 'i', '.']
['s', 'h', '.']
['a', 'n', 'a', 'n', '.']
['d', 'b', 'd', 'a', 'n', 'i', 'r', 'w', 'i', '.']
['s', 'h', '.']
['s', 'h', 'i', '.']
['a', 'n', 'a', 'n', '.']
['a', '.']
['s', 'h', '.']
['s', 'h', 'v', 't', 'u', 'm', 't', 'r', 'a', 'n', '.']
['s', 'h', 'u', 'm', 'j', 'm', 'p', 'p', 'p', 'a', 's', 'h', 'i', '.']
['a', 'n', 'u', 'a', 'n', 'i', '.']
['s', 'o', 'r', 'a', '.']
['y', 'a', '.']
['s', 'h', 'a', 'n', 'a', 'n', '.']
['a', 'n', 'a', 'n', 'i', '.']
['s', 'h', 'i', 'm', 'o', 'i', '.']
['m', 'w', 'y', 'f', 'v', 'n', '.']
['a', 'n', 'a', 'n', 's', 'n', 'h', 'a', 'v', 'j', 'i', 't', 's', 'h', 'a', '.']
['s', 'h', 'a', 'd', 'g', 'o', 'n', 'f', 'm', 'p', 'a', 'n', 'a', 'n', 'i', '.']
['s', 'h', '.']
